<a href="https://colab.research.google.com/github/a6iyyu/MachineLearning_3F_23/blob/main/JS6/P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 2**

Pada percobaan kali ini kita akan membuat pencarian sederhana menggunakan FAISS. Install FAISS terlebih dahulu menggunakan FAISS-CPU, jika ingin menggunakan GPU, dapat lakukan instalasi FAISS-GPU.

In [10]:
import numpy as np
import pandas as pd
import faiss
import time

In [11]:
def run_experiment(n_points, dim, metric_type, k=10):
    """Fungsi untuk menjalankan satu skenario eksperimen."""
    print(f"--- Menjalankan Eksperimen: {n_points} data, {dim}D, Metrik {metric_type} ---")

    # 1. Buat dataset
    X = np.random.rand(n_points, dim).astype('float32')
    query = np.random.rand(1, dim).astype('float32')

    if metric_type == 'IP':
        # Untuk Inner Product, normalisasi vektor adalah praktik standar
        # agar setara dengan cosine similarity.
        faiss.normalize_L2(X)
        faiss.normalize_L2(query)

    # 2. Exact NN (Brute-force)
    if metric_type == 'L2':
        index_exact = faiss.IndexFlatL2(dim)
    else: # IP
        index_exact = faiss.IndexFlatIP(dim)

    index_exact.add(X)

    start_time = time.time()
    D_exact, I_exact = index_exact.search(query, k)
    time_exact = time.time() - start_time

    # 3. Approximate NN (IVF+PQ)
    # Tentukan parameter IVF+PQ. Untuk data besar, nlist harus lebih besar.
    nlist = 100 if n_points > 10000 else 10 # Jumlah cluster
    m = dim if dim <= 8 else 8 # Jumlah sub-kuantizer, tidak boleh lebih besar dari dim
    bits = 8 # bit per sub-vektor

    if metric_type == 'L2':
        quantizer = faiss.IndexFlatL2(dim)
        index_approx = faiss.IndexIVFPQ(quantizer, dim, nlist, m, bits)
    else: # IP
        quantizer = faiss.IndexFlatIP(dim)
        index_approx = faiss.IndexIVFPQ(quantizer, dim, nlist, m, bits)

    # Training & adding data
    index_approx.train(X)
    index_approx.add(X)

    # Searching
    index_approx.nprobe = 5 # Berapa banyak cluster terdekat yang akan dicari
    start_time = time.time()
    D_approx, I_approx = index_approx.search(query, k)
    time_approx = time.time() - start_time

    # 4. Hitung recall
    exact_neighbors = set(I_exact[0])
    approx_neighbors = set(I_approx[0])
    recall = len(exact_neighbors.intersection(approx_neighbors)) / k

    # 5. Hitung speedup
    speedup = time_exact / time_approx if time_approx > 0 else float('inf')

    return {
        "Jumlah Data": n_points,
        "Dimensi": dim,
        "Metrik Jarak": metric_type,
        "Waktu Exact (ms)": time_exact * 1000,
        "Waktu IVF+PQ (ms)": time_approx * 1000,
        "Speedup": speedup,
        "Recall@10": recall
    }

In [12]:
# Daftar skenario eksperimen
experiments = [
    {'n_points': 1000, 'dim': 2, 'metric_type': 'L2'},
    {'n_points': 1000, 'dim': 2, 'metric_type': 'IP'},
    {'n_points': 1000, 'dim': 5, 'metric_type': 'L2'},
    {'n_points': 1000, 'dim': 5, 'metric_type': 'IP'},
    {'n_points': 1_000_000, 'dim': 2, 'metric_type': 'L2'},
    {'n_points': 1_000_000, 'dim': 2, 'metric_type': 'IP'},
    {'n_points': 1_000_000, 'dim': 5, 'metric_type': 'L2'},
    {'n_points': 1_000_000, 'dim': 5, 'metric_type': 'IP'},
]

results = []
for params in experiments:
    result = run_experiment(**params)
    results.append(result)

--- Menjalankan Eksperimen: 1000 data, 2D, Metrik L2 ---
--- Menjalankan Eksperimen: 1000 data, 2D, Metrik IP ---
--- Menjalankan Eksperimen: 1000 data, 5D, Metrik L2 ---
--- Menjalankan Eksperimen: 1000 data, 5D, Metrik IP ---
--- Menjalankan Eksperimen: 1000000 data, 2D, Metrik L2 ---
--- Menjalankan Eksperimen: 1000000 data, 2D, Metrik IP ---
--- Menjalankan Eksperimen: 1000000 data, 5D, Metrik L2 ---
--- Menjalankan Eksperimen: 1000000 data, 5D, Metrik IP ---


Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [13]:
# Tampilkan hasil dalam bentuk tabel
df = pd.DataFrame(results)
print("\n--- HASIL EKSPERIMEN ---")
print(df.to_string())


--- HASIL EKSPERIMEN ---
   Jumlah Data  Dimensi Metrik Jarak  Waktu Exact (ms)  Waktu IVF+PQ (ms)    Speedup  Recall@10
0         1000        2           L2          0.056744           0.085592   0.662953        1.0
1         1000        2           IP          0.038385           0.069380   0.553265        0.9
2         1000        5           L2          0.024796           0.132561   0.187050        1.0
3         1000        5           IP          0.029325           0.109911   0.266811        1.0
4      1000000        2           L2          5.857468           0.324011  18.077999        0.9
5      1000000        2           IP          9.225130           0.307322  30.017843        0.1
6      1000000        5           L2          6.485224           0.583410  11.116060        1.0
7      1000000        5           IP         12.939692           0.540257  23.951015        1.0
